In [1]:
import requests
import pandas as pd
import time
import json
import ast
import numpy as np
import glob
import datetime

In [2]:
from ta import trend
from ta import momentum

In [ ]:
def convert_to_json(text):
    try:
        return ast.literal_eval(text)
    except:
#         print(text)
        return {}

In [ ]:
def read_all_file_in_directory(directory, file_type, sep = "\t"):
    print(directory)
    file_list= [f for f in glob.glob(directory + "*." + file_type)]
    
    all_data = pd.DataFrame()
    temp = pd.DataFrame()
    for file in file_list:
            df = pd.read_csv(file, sep= sep, encoding = "utf8", engine='python')
            temp = pd.concat([temp, df])
    all_data = pd.concat([all_data, temp])
    return all_data

In [ ]:
# def clean_candlestick_data(df, price_type = 'ask' ):
#     df_modified = df[[ 'currency_pair', 'time', 'volume', price_type]]
    
#     df_modified[price_type] = df_modified[price_type].apply(convert_to_json)
#     price_df = pd.json_normalize(df_modified[price_type])
    
#     df_modified = pd.merge(df_modified, price_df,
#                           left_index=True, right_index=True)
    
#     df_modified = df_modified[['currency_pair', 'time', 'volume','o', 'h', 'l', 'c']]
#     df_modified.columns = ['currency_pair', 'time', 'volume','price_open','price_high', 'price_low', 'price_close']
    
#     df_modified["price_open"] = df_modified["price_open"].astype(float)
#     df_modified["price_high"] = df_modified["price_high"].astype(float)
#     df_modified["price_low"] = df_modified["price_low"].astype(float)
#     df_modified["price_close"] = df_modified["price_close"].astype(float)
#     df_modified["time"] = pd.to_datetime(df_modified["time"])
    
#     return(df_modified)


def clean_candle_data(df, price_type = 'ask' ):
    
    df_modified = df[['complete', 'currency_pair', 'time', 'volume', price_type]]
    df_modified[price_type] = df_modified[price_type].astype(str)
    df_modified[price_type] = df_modified[price_type].apply(convert_to_json)
    
    price_df = pd.json_normalize(df_modified[price_type])
    
    df_modified = pd.merge(df_modified, price_df,
                          left_index=True, right_index=True)
    print(df_modified.columns)
    
    df_modified = df_modified[['complete','currency_pair', 'time', 'volume','o', 'h', 'l', 'c']]
    df_modified.columns = ['complete','currency_pair', 'time', 'volume','price_open','price_high', 'price_low', 'price_close']
    
    df_modified[["price_open", "price_high", "price_low", "price_close"]] = df_modified[["price_open", "price_high",
                                                                                         "price_low", "price_close"]].apply(pd.to_numeric)
    df_modified["time"] = pd.to_datetime(df_modified["time"])
    
    return df_modified

In [ ]:
def get_macd_indicator(df, window_slow: int = 26, window_fast: int = 12, window_sign: int = 9, fillna: bool = False):
    # Innitialising MACD indicator
    indicator_macd = trend.MACD(close = df["price_close"])
    # Create columns for MACCD line, singal line and macd histogram (macd crosses signal line)
    df['macd'] = indicator_macd.macd()
    df['macd_signal'] = indicator_macd.macd_signal()
    df['macd_signal_diff'] = indicator_macd.macd_diff()
    return df

In [ ]:
"""
Create an empty dictionary to store time of potential trades (buy and sell)
If the dictionary is empty AND
if the ccurrent macd_signal_diff value is NOT nan then
store the fist non-nan value
"""
"""
If the dictionary is NOT empty AND
if the current macd_signal_diff is > 0 AND
if the previous macd_signal_diff is < 0 THEN
store the current macd_signal_diff value
"""
"""
If the dictionary is NOT empty AND
if the current macd_signal_diff is < 0 AND
if the previous macd_signal_diff is > 0 THEN
store the current macd_signal_diff value
"""

def macd_buy_sell_signal(df):
    
    df = get_macd_indicator(df)
    
    macd_signal_dict = {}
    for i in range(0,len(df)):
        if bool(macd_signal_dict) == False:
            if np.isnan(df["macd_signal_diff"][i]) == False: 
                macd_signal_dict[df["time"][i]] = df["macd_signal_diff"][i]    
        else:
            if df["macd_signal_diff"][i] > 0:
                if list(macd_signal_dict.values())[-1] < 0:
                    macd_signal_dict[df["time"][i]] = df["macd_signal_diff"][i]

            elif df["macd_signal_diff"][i] < 0:
                if list(macd_signal_dict.values())[-1] > 0:
                    macd_signal_dict[df["time"][i]] = df["macd_signal_diff"][i]

    macd_signal_df= pd.DataFrame(macd_signal_dict.items())
    macd_signal_df.columns = ["time", "macd_signal_value"]
    macd_signal_df['buy_sell_signal'] = np.where(macd_signal_df['macd_signal_value'] > 0, "buy", "sell")
    
    df = pd.merge(df, macd_signal_df,
                      on ="time",
                      how = "left")
    return df              

In [ ]:
def trade_execution(df, order_type, macd_signal_time, take_profit_pip = 10, stop_loss_pip = 10):
    
    df["index"] = df.index
    signal_time_index = df.loc[df['time'] == macd_signal_time, 'index'].iloc[0]
    order_time_index = signal_time_index + 1
    order_price_start = df["price_open"][order_time_index]
    order_time_start = df["time"][order_time_index]

    if order_type == "buy":
        price_tp = round(order_price_start + (take_profit_pip/10000),5)
        price_sl = round(order_price_start - (stop_loss_pip/10000),5)
#         print(order_type, order_time_start, "start:" + str(order_price_start), "tp:" + str(price_tp), "sl:" + str(price_sl) )
        for index in range(order_time_index, len(df)):
            current_price_low = round(df["price_low"][index],5)
            current_price_high = round(df["price_high"][index],5)
            current_time = df["time"][index]
#             print(order_type, current_time, "start:" + str(order_price_start), "low:" + str(current_price_low), "high:" + str(current_price_high) )
            if (current_price_low <= price_sl) and (current_price_high >= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Not Determined"}, index=[0])
                return result_df
                break
            elif (current_price_low <= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Stop Loss Order Executed"}, index=[0])
                return result_df
                break
            elif (current_price_high >= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Take Profit Order Executed"}, index=[0])
                return result_df
                break
            else:
                continue

    elif order_type == "sell":
        price_tp = round(order_price_start - (take_profit_pip/10000),5)
        price_sl = round(order_price_start + (stop_loss_pip/10000),5)
#         print(order_type, order_time_start, "start:" + str(order_price_start), "tp:" + str(price_tp), "sl:" + str(price_sl) )

        for index in range(order_time_index, len(df)):
            current_price_low = round(df["price_low"][index],5)
            current_price_high = round(df["price_high"][index],5)
            current_time = df["time"][index]
#             print(order_type, current_time, "start:" + str(order_price_start), "low:" + str(current_price_low), "high:" + str(current_price_high) )

            if (current_price_low <= price_tp) and (current_price_high >= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Not Determined"}, index=[0])
                return result_df
                break
            elif (current_price_low <= price_tp):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Take Profit Order Executed"}, index=[0])
                return result_df
                break
            elif (current_price_high >= price_sl):
                result_df = pd.DataFrame({'order_type': order_type,
                                          'order_time_start': order_time_start,
                                          'order_time_end': current_time,
                                          'order_price_start': order_price_start,
                                          'order_price_sl': price_sl,
                                          'order_price_tp': price_tp,
                                          'current_price_high': current_price_high,
                                          'current_price_low': current_price_low,
                                          'order_outcome': "Stop Loss Order Executed"}, index=[0])
                return result_df
                break
            else:
                continue
    else:
        return("Invalid Order Type (only buy or sell)")

    

#### READ AND CLEAN PRICE DATA

In [ ]:
directory = "/Users/nguyenhoangnam/Documents/Python/Stock - Forex Investing & Trading/Forex/OANDA/"
curency_pair = "EURUSD"
time_range = "M15"
path = directory + curency_pair + "/" +  time_range + "/Raw Data/"


df = read_all_file_in_directory(path, file_type = "csv")
df.shape

In [ ]:
df_clean = clean_candlestick_data(df)

In [ ]:
df

#### STRATERGY 1: MACD CROSSOVER

In [ ]:
# Get MACD buy and sell signals
df_with_macd_signals = macd_buy_sell_signal(df_clean)

In [ ]:
indicator_sma = trend.SMAIndicator(close = df_with_macd_signals["price_close"], window=200)
df_with_macd_signals['sma_200'] = indicator_sma.sma_indicator()

In [ ]:
indicator_rsi = momentum.RSIIndicator(close = df_with_macd_signals["price_close"], window=14)
df_with_macd_signals['rsi_14'] = indicator_rsi.rsi()

In [ ]:
def filter_trade_signals(df, rsi_oversold, rsi_overbought):
    df["new_buy_sell_signal"] = ""
    for i in range(len(df)-3):
        if (df["buy_sell_signal"][i] == "buy") and (df["macd_signal_diff"][i+1] > 0)  and (df["price_close"][i+1] > df["sma_200"][i+1]) and (df["rsi_14"][i+1] < rsi_overbought):
            df["new_buy_sell_signal"][i+1] = "buy"
        elif (df["buy_sell_signal"][i] == "sell") and (df["macd_signal_diff"][i+1] < 0) and (df["price_close"][i+1] < df_with_macd_signals["sma_200"][i+1]) and (df["rsi_14"][i+1] > rsi_oversold):
            df["new_buy_sell_signal"][i+1] = "sell"
#     print(df["buy_sell_signal"].value_counts())
#     print(df["new_buy_sell_signal"].value_counts())
    return df
            

In [ ]:
def get_trade_results(df):

    df_with_macd_signals_filtered = df[df["new_buy_sell_signal"].isin(["buy", "sell"])].reset_index(drop=True)
    df_with_macd_signals_filtered.shape

    

    output_df = pd.DataFrame()
    for i in range(0,len(df_with_macd_signals_filtered)-1):
        output = trade_execution(df, df_with_macd_signals_filtered["new_buy_sell_signal"][i],
                                df_with_macd_signals_filtered["time"][i], 10,10)
        output_df = output_df.append(output)
    return output_df
    



     

In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
results_df = []
for oversold_value in range(25,100,5):
    for overbought_value in range(75,0,-5):
        
        df_with_macd_signals = filter_trade_signals(df_with_macd_signals, oversold_value,overbought_value)
        output_df = get_trade_results(df_with_macd_signals)
        total_trade = len(output_df)
        try:
          winner_count = len(output_df[output_df["order_outcome"] == "Take Profit Order Executed"])
        except:
          winner_count = 0
        try:
          loser_count = len(output_df[output_df["order_outcome"] == "Stop Loss Order Executed"])
        except:
          loser_count = 0
        try:
          win_rate = winner_count/total_trade*100
        except:
          win_rate = 0
        
        print(oversold_value, overbought_value,win_rate,total_trade)
        temp = pd.DataFrame(
            {'oversold_value': oversold_value, 'overbought_value': overbought_value,
             'win_rate': win_rate, 'total_trade': total_trade, 'winner_count': winner_count, 'loser_count': loser_count}, index=[0])
        results_df.append(temp)
results_df = pd.concat(results_df, ignore_index=True) 

In [ ]:
results_df["profit"] = (results_df["win_rate"] - 50) * results_df["total_trade"]

In [ ]:
results_df = results_df.sort_values("profit", ascending= False)

In [ ]:
results_df['currency_pair'] = curency_pair
results_df['time_range'] = time_range

In [ ]:
results_df

In [ ]:
df.to_csv("/Users/nguyenhoangnam/Downloads/2010-01-01_2021-05-01_eur_usd_m15_strategy_1_win_rates.csv", encoding="utf8", index=None, sep="\t")

In [ ]:
results_df.head(50)

In [ ]:
reviews_df["scenario_number"] = reviews_df.index +1

In [ ]:
test = reviews_df[reviews_df["win_rate"] == reviews_df["win_rate"].max()]

In [ ]:
test

In [ ]:
scenario_number_list = reviews_df["scenario_number"].to_list()
win_rate_list = reviews_df["scenario_number"].to_list()

In [ ]:
scenario_number

In [ ]:
import matplotlib.pyplot as plt

plt.plot(reviews_df["scenario_number"],reviews_df["win_rate"])
plt.title('title name')
plt.xlabel('xAxis name')
plt.ylabel('yAxis name')
plt.show()

In [ ]:
df_with_macd_signals["buy_sell_signal"].value_counts()

In [ ]:
df_with_macd_signals["new_buy_sell_signal"].value_counts()

In [ ]:
df_with_macd_signals_filtered = df_with_macd_signals[df_with_macd_signals["new_buy_sell_signal"].isin(["buy", "sell"])].reset_index(drop=True)
df_with_macd_signals_filtered.shape

In [ ]:
# trade at [i]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+2]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+3]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+4]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+5]
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i] and above/below SMA200
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] and above/below SMA200
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+2] and above/below SMA200
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (30, 70)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (35, 65)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (40, 60)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
# trade at [i+1] AND above/below SMA200 AND RSI (40, 60)
len(df[df["order_outcome"] == "Take Profit Order Executed"])/len(df)*100

In [ ]:
df_with_macd_signals["new_buy_sell_signal"] = ""
for i in range(len(df_with_macd_signals)-3):
    if (df_with_macd_signals["buy_sell_signal"][i] == "buy") and (df_with_macd_signals["macd_signal_diff"][i+1] > 0)  and (df_with_macd_signals["price_close"][i+1] > df_with_macd_signals["sma_200"][i+1]) and (df_with_macd_signals["rsi_14"][i+1] < 60):
        df_with_macd_signals["new_buy_sell_signal"][i+1] = "buy"
    elif (df_with_macd_signals["buy_sell_signal"][i] == "sell") and (df_with_macd_signals["macd_signal_diff"][i+1] < 0) and (df_with_macd_signals["price_close"][i+1] < df_with_macd_signals["sma_200"][i+1]) and (df_with_macd_signals["rsi_14"][i+1] > 40):
        df_with_macd_signals["new_buy_sell_signal"][i+1] = "sell"
        

In [ ]:
df["order_outcome"].value_counts()

In [ ]:
pd.DataFrame({"count": df.groupby(["order_type", "order_outcome"]).size()}).reset_index().sort_values("order_type", ascending = False)
